**Goal:** Demonstrate your ability to program a RAG system that uses BigQuery to generate embeddings and perform a vector search.

**Requirements:**


*  Students are provided with a set of frequently asked questions for the fictitious town of Aurora Bay, Alaska
*   They will need to load the data into a BigQuery table.
*  Students need to create embeddings for each record and store them in BigQuery.
*  Implement a chatbot that can search the BigQuery embeddings to answer user questions accurately using the proprietary data.

In [1]:
pip install google-cloud-bigquery google-genai

In [2]:
import os

import google.auth
from google.cloud import bigquery

from google import genai
from google.genai import types

import vertexai


In [3]:
credentials, project_id = google.auth.default()

DATASET_ID = "rag_dataset"
TABLE_ID = "faqs"
DATA_URL = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"
FULL_TABLE_ID = f"{project_id}.{DATASET_ID}.{TABLE_ID}"
print(FULL_TABLE_ID)

qwiklabs-gcp-01-c72d7cb996a1.rag_dataset.faqs


In [4]:
client = bigquery.Client(project=project_id)

dataset_ref = client.dataset(DATASET_ID)
try:
    client.get_dataset(dataset_ref)
except Exception:
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    client.create_dataset(dataset)

In [5]:
# set up dataset
schema = [
    bigquery.SchemaField("question", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("answer", "STRING", mode="NULLABLE")
]

job_config = bigquery.LoadJobConfig(
    schema=schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows = 1,
)

load_job = client.load_table_from_uri(
    source_uris=DATA_URL,
    destination=FULL_TABLE_ID,
    job_config=job_config
)

In [6]:
load_job.result()

LoadJob<project=qwiklabs-gcp-01-c72d7cb996a1, location=US, id=9f6dc839-c250-40b6-aecc-8ca545f01ae7>

# Generate Embeddings

In [7]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
# creating a connection between model (vertex ai) and bigquery
CREATE OR REPLACE MODEL
  `rag_dataset.embedding_model`
REMOTE WITH CONNECTION
  `us.embedding_conn`
OPTIONS (ENDPOINT = 'text-embedding-005');


""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [8]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
# grab and concat each question and answer from FAQs
# generate embeddings
# store them in table
CREATE OR REPLACE TABLE
  `rag_dataset.faq_with_embeddings` AS
SELECT *
FROM ML.GENERATE_EMBEDDING(
    MODEL `rag_dataset.embedding_model`,
    (
      SELECT
        question,
        answer,
        CONCAT('Question: ', question, ' Answer ', answer) AS content
      FROM `rag_dataset.faqs`)
);
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [9]:
def do_search(user_prompt: str):
  # SQL lookup
  query = f"""
  SELECT
    base.question,
    base.answer,
    distance
  FROM
    VECTOR_SEARCH(
      TABLE `rag_dataset.faq_with_embeddings`,
      'ml_generate_embedding_result',
      (
        SELECT ml_generate_embedding_result, content AS query
        FROM ML.GENERATE_EMBEDDING(
          MODEL `rag_dataset.embedding_model`,
          (SELECT '{user_prompt}' AS content)
        )
      ),
      top_k => 3
    );
  """

  results = client.query(query).to_dataframe()
  return results

# Setting up LLM stuff

In [20]:
GEN_MODEL = "gemini-2.5-flash"
SYS_PROMPT = """System Prompt:
You are a chatbot who's main purpose is to answer questions related to the town of Aurora Bay
Respond with information that only comes from the context provided to you
If you do not know the answer, do not make something up
"""

In [21]:
credentials, project_id = google.auth.default()
vertexai.init(project=project_id, location="us-east1")

In [12]:
def generate(user_prompt: str):
  client = genai.Client(
      vertexai=True,
      api_key=os.environ.get("GOOGLE_CLOUD_API_KEY"),
  )

  model = GEN_MODEL
  contents = [
    types.Content(
      role="user",
      parts=[types.Part.from_text(text=user_prompt)]
    )
  ]
  tools = [
    types.Tool(google_search=types.GoogleSearch()),
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 65535,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_ONLY_HIGH"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_ONLY_HIGH"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_ONLY_HIGH"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_ONLY_HIGH"
    )],
    tools = tools,
    system_instruction=[types.Part.from_text(text=SYS_PROMPT)],
    thinking_config=types.ThinkingConfig(
      thinking_budget=0,
    ),
  )

  full_resp_parts = []
  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    if not chunk.candidates or not chunk.candidates[0].content or not chunk.candidates[0].content.parts:
        continue

    #print(chunk.text, end="")
    full_resp_parts.append(chunk.text)

  full_resp = "".join(full_resp_parts)
  return full_resp


In [16]:
RAG_TEMPLATE = """
Context:
---
{search_results}
---

Question:
{user_prompt}

Answer the question based ONLY on the provided context if possible. If the context does not contain the answer, state that you could not find the answer in the provided context.
"""

In [14]:
def generate_with_rag(user_prompt: str):
    results_df = do_search(user_prompt)

    # rows to strings
    formatted_results = []
    for index, row in results_df.iterrows():
        # You can choose which columns to include. `question` and `answer` are key.
        formatted_results.append(
            f"Q: {row['question']}\n"
            f"A: {row['answer']}\n"
            f"Distance: {row['distance']}"
        )
    search_results_text = "\n\n".join(formatted_results)

    # stick prompt and search results into template
    final_prompt = RAG_TEMPLATE.format(
        search_results=search_results_text,
        user_prompt=user_prompt
    )

    # gen response
    llm_response = generate(final_prompt)

    return llm_response

In [24]:
user_prompt = "Tell me about the town hall in Aurora"

resp = generate_with_rag(user_prompt=user_prompt)
print(resp)


The Aurora Bay Town Hall is located at 100 Harbor View Road, in the center of Aurora Bay, near the main harbor.


In [23]:
user_prompt = "Tell me about the public transit in Aurora?"
resp = generate_with_rag(user_prompt=user_prompt)
print(resp)

Aurora Bay offers a limited bus service on weekdays. The service runs from 6 AM to 8 PM and covers major routes, including downtown, the airport, and residential neighborhoods.
